1. Authentifications et token

Objectif : comprendre les mécanismes d'authentification les plus courants côté API.

Notions clés :
- API key : une clé statique transmise dans un header ou un paramètre.
- Bearer token : un jeton transmis dans l'en-tête Authorization.
- Bonnes pratiques : stocker le secret dans une variable d'environnement.

Exemple d'en-tête HTTP :
```
Authorization: Bearer MON_TOKEN_SECRET
```

In [ ]:
import os

# Récupère un token depuis les variables d'environnement.
# Cela évite de coder le secret en dur dans le notebook.
TOKEN = os.getenv('API_TOKEN', 'token_de_demo')

# Construit un dictionnaire d'en-têtes HTTP.
headers = {
    # Authorization est l'en-tête standard pour les tokens Bearer.
    'Authorization': f'Bearer {TOKEN}',
    # Content-Type indique le format des données envoyées.
    'Content-Type': 'application/json',
}

# Affiche les headers pour la démonstration.
print(headers)

2. Anatomie d’une API REST

Objectif : visualiser les éléments essentiels d'une requête REST.

Éléments principaux :
- Méthode HTTP : GET, POST, PUT, DELETE.
- URL : domaine + ressource + paramètres.
- Headers : métadonnées (auth, type, cache).
- Body : contenu JSON pour créer/modifier.

Exemple d'URL :
```
https://api.exemple.com/v1/tweets?limit=10&lang=fr
```

In [ ]:
from urllib.parse import urlparse, parse_qs

# URL d'exemple à analyser.
url = 'https://api.exemple.com/v1/tweets?limit=10&lang=fr'

# Analyse l'URL en ses différentes parties.
parsed = urlparse(url)

# Affiche le schéma (http/https).
print('schema:', parsed.scheme)
# Affiche le domaine.
print('domaine:', parsed.netloc)
# Affiche le chemin de la ressource.
print('chemin:', parsed.path)
# Affiche les paramètres de requête.
print('params:', parse_qs(parsed.query))

3. Retry propre

Objectif : retenter proprement une requête temporairement en échec.

Bonnes pratiques :
- Utiliser un backoff exponentiel.
- Limiter le nombre de tentatives.
- Retenter uniquement sur des erreurs temporaires (ex: 429, 503).

In [ ]:
import time

# Fonction de retry générique pour un appel réseau simulé.
def retry_avec_backoff(operation, max_tentatives=3, base_delay=0.5):
    # Boucle sur les tentatives.
    for tentative in range(1, max_tentatives + 1):
        try:
            # Exécute l'opération et retourne le résultat si succès.
            return operation()
        except Exception as exc:
            # Si on a épuisé les tentatives, on relance l'erreur.
            if tentative == max_tentatives:
                raise
            # Calcule un délai exponentiel simple.
            delay = base_delay * (2 ** (tentative - 1))
            # Affiche une info de retry pour l'utilisateur.
            print(f'Retry {tentative}/{max_tentatives} après {delay:.2f}s: {exc}')
            # Attends avant de retenter.
            time.sleep(delay)

# Démonstration avec une fonction instable.
compteur = {'n': 0}

def operation_instable():
    # Incrémente le compteur d'appels.
    compteur['n'] += 1
    # Échoue sur les deux premiers appels.
    if compteur['n'] < 3:
        raise RuntimeError('Erreur temporaire')
    # Renvoie un résultat à la 3e tentative.
    return 'OK'

# Lance l'opération avec retry.
print(retry_avec_backoff(operation_instable))

4. Gérer le rate limiting

Objectif : respecter les limites imposées par un service.

Exemple simple : laisser passer N requêtes par fenêtre de temps.

In [ ]:
import time

# Limiteur de débit très simple basé sur un intervalle minimal.
class SimpleRateLimiter:
    # max_par_seconde définit la cadence autorisée.
    def __init__(self, max_par_seconde):
        # Intervalle minimal entre deux appels.
        self.min_interval = 1.0 / max_par_seconde
        # Timestamp du dernier appel.
        self._last_call = 0.0

    # Méthode à appeler avant chaque requête.
    def wait(self):
        # Calcule le temps écoulé depuis le dernier appel.
        elapsed = time.time() - self._last_call
        # Si trop rapide, on attend le temps restant.
        if elapsed < self.min_interval:
            time.sleep(self.min_interval - elapsed)
        # Met à jour le dernier timestamp.
        self._last_call = time.time()

# Démonstration du limiter avec 2 requêtes par seconde.
limiter = SimpleRateLimiter(max_par_seconde=2)
for i in range(3):
    # Attend si nécessaire pour respecter la cadence.
    limiter.wait()
    # Affiche l'instant de la requête simulée.
    print('Requête', i, 'à', round(time.time(), 2))

5. Gestion des erreurs

Objectif : distinguer les erreurs client, serveur et réseau.

Stratégie :
- 4xx : erreur côté client (requête invalide).
- 5xx : erreur côté serveur (retry possible).
- Réseau : timeout, DNS, etc.

In [ ]:
# Définition d'exceptions spécialisées.
class ApiError(Exception):
    pass

class ClientError(ApiError):
    pass

class ServerError(ApiError):
    pass

# Exemple de fonction qui traite un statut HTTP.
def handle_status(status_code):
    # 400-499 : erreurs client.
    if 400 <= status_code < 500:
        raise ClientError(f'Client error: {status_code}')
    # 500-599 : erreurs serveur.
    if 500 <= status_code < 600:
        raise ServerError(f'Server error: {status_code}')
    # Sinon, on considère que tout va bien.
    return 'OK'

# Démonstration simple.
try:
    print(handle_status(404))
except ApiError as exc:
    print('Erreur capturée:', exc)

6. Logging de l’application

Objectif : tracer les actions clés pour diagnostiquer les problèmes.

Points clés :
- Niveaux : DEBUG, INFO, WARNING, ERROR.
- Format lisible et horodaté.
- Éviter de loguer des secrets.

In [ ]:
import logging

# Configure le logger principal.
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Crée un logger nommé pour le module API.
logger = logging.getLogger('api_client')

# Exemple de logs aux différents niveaux.
logger.debug('Message de debug (souvent caché).')
logger.info('Client initialisé.')
logger.warning('Utilisation d’un token de démo.')
logger.error('Exemple d’erreur (sans exception).')

7. Exemple avec un client twitter fait à la main

Objectif : assembler un mini-client HTTP complet, sans dépendance externe.

L'exemple ci-dessous simule un service Twitter avec des réponses prédéfinies.
Cela permet de tester la logique sans appeler un vrai réseau.

In [ ]:
import json
import time
import logging

# Représente une réponse HTTP minimale.
class FakeResponse:
    # status_code : code HTTP (200, 404, 429, ...)
    # data : dictionnaire qui représente le JSON retourné
    def __init__(self, status_code, data):
        self.status_code = status_code
        self._data = data

    # Simule la méthode json() d'un client HTTP classique.
    def json(self):
        return self._data

# Transport fictif qui renvoie des réponses prédéfinies.
class FakeTransport:
    def __init__(self):
        # Liste de réponses successives pour simuler des erreurs temporaires.
        self.responses = [
            FakeResponse(429, {'error': 'rate limited'}),
            FakeResponse(503, {'error': 'temporary'}),
            FakeResponse(200, {'id': 1, 'text': 'Bonjour API'}),
        ]

    # Simule un envoi de requête.
    def send(self, method, url, headers, body=None):
        # Si on a encore des réponses en attente, on renvoie la suivante.
        if self.responses:
            return self.responses.pop(0)
        # Sinon, on renvoie une réponse stable.
        return FakeResponse(200, {'id': 2, 'text': 'Autre tweet'})

# Client API minimaliste avec auth, retry, rate limit et erreurs.
class TwitterClient:
    def __init__(self, base_url, token, transport, rate_limiter):
        # URL de base de l'API.
        self.base_url = base_url
        # Token d'authentification.
        self.token = token
        # Transport (réel ou simulé).
        self.transport = transport
        # Limiteur de débit.
        self.rate_limiter = rate_limiter
        # Logger dédié au client.
        self.logger = logging.getLogger('twitter_client')

    # Construit les headers communs.
    def _headers(self):
        return {
            'Authorization': f'Bearer {self.token}',
            'Content-Type': 'application/json',
        }

    # Envoie une requête avec retry et gestion d'erreur.
    def request(self, method, path, body=None, max_retries=3):
        # Construit l'URL complète.
        url = f'{self.base_url}{path}'
        # Boucle de retry.
        for attempt in range(1, max_retries + 1):
            # Respecte le rate limit avant chaque appel.
            self.rate_limiter.wait()
            # Journalise la requête.
            self.logger.info('Requête %s %s (tentative %d)', method, url, attempt)
            # Envoie la requête via le transport.
            response = self.transport.send(method, url, self._headers(), body)
            # Traite les statuts.
            if response.status_code == 429:
                # Rate limit : on attend un peu et on retente.
                self.logger.warning('Rate limited, retry...')
                time.sleep(0.5)
                continue
            if 500 <= response.status_code < 600:
                # Erreur serveur temporaire.
                self.logger.warning('Erreur serveur, retry...')
                time.sleep(0.5)
                continue
            if 400 <= response.status_code < 500:
                # Erreur client : on ne retry pas.
                raise ClientError(f'Client error: {response.status_code}')
            # Succès : on retourne le JSON.
            return response.json()
        # Si on arrive ici, toutes les tentatives ont échoué.
        raise ServerError('Échec après retries')

    # Exemple d'appel API pour lire un tweet.
    def get_tweet(self, tweet_id):
        # Appelle la route GET /tweets/{id}.
        return self.request('GET', f'/tweets/{tweet_id}')

# Prépare les composants du client.
transport = FakeTransport()
rate_limiter = SimpleRateLimiter(max_par_seconde=5)
client = TwitterClient(
    base_url='https://api.fake-twitter.local',
    token=TOKEN,
    transport=transport,
    rate_limiter=rate_limiter,
)

# Appelle le client et affiche la réponse.
print(client.get_tweet(1))

8. Exemple avec un vrai client HTTP (requests)

Objectif : faire une vraie requête HTTP avec un client standard.

Points clés :
- Utiliser un timeout court pour éviter les blocages.
- Vérifier le code de statut et traiter le JSON.
- Garder les headers d'authentification si nécessaire.

L'exemple ci-dessous interroge un service public de test (JSONPlaceholder).

In [ ]:
import requests

# URL réelle d'un service public de démonstration.
real_url = 'https://jsonplaceholder.typicode.com/posts/1'

# Construit les headers, ici on réutilise Authorization pour l'exemple.
real_headers = {
    'Authorization': f'Bearer {TOKEN}',
    'Accept': 'application/json',
}

# Envoie une requête GET avec un timeout court.
response = requests.get(real_url, headers=real_headers, timeout=5)

# Vérifie que la requête a réussi (code 200-299).
response.raise_for_status()

# Parse le JSON de la réponse.
payload = response.json()

# Affiche quelques champs pour la démonstration.
print('id:', payload.get('id'))
print('title:', payload.get('title'))

8. Exercices finaux

1. Ajouter un cache local pour éviter de re-télécharger le même tweet.
2. Ajouter un timeout simulé et tester la logique de retry.
3. Étendre le client pour publier un tweet (POST /tweets).
4. Ajouter un mécanisme de pagination.
5. Enregistrer les logs dans un fichier.
